In [34]:
from typing import List, Optional
from dataclasses import dataclass
from steering_metrics.factories import get_big_query_client
from arrow import Arrow
import pandas as pd

In [35]:
start_period = Arrow(2026, 1, 12,timezone="utc")
end_period = start_period.shift(days=7)
EXCLUDED_ENERGY_SUPPLIER = "Frank Energie"

In [36]:
@dataclass
class SteeredDevice:
    device_id: str
    user_id: str
    brand: str
    system_id: str
    energy_supplier: str
    dates: Optional[List[Arrow]] = None

In [41]:
def get_unique_steered_devices_for_day(date: Arrow) -> pd.DataFrame:
    partition_date = date.datetime.strftime('%Y-%m-%d')
    query =  f"""
        SELECT
          DISTINCT device_id,
                   user_id,
                   brand,
                   system_id,
                   energy_supplier
        FROM `nox-energy-431214.asset_telemetry_raw.rbc_decisions`
        WHERE partition_date >= DATE('{partition_date}')
            AND energy_supplier != '{EXCLUDED_ENERGY_SUPPLIER}'
        ORDER BY brand, device_id
    """
    query_job = get_big_query_client().query(query)
    response = query_job.result()
    data = {}
    for row in response:
        # data.append(dict(row))
        data[dict(row)["device_id"]] = dict(row)
    return data

In [42]:
def get_unique_steered_devices_for_period(start: Arrow, end: Arrow) -> pd.DataFrame:
    partition_date_start = start.datetime.strftime('%Y-%m-%d')
    partition_date_end = end.datetime.strftime('%Y-%m-%d')
    query =  f"""
        SELECT
          DISTINCT device_id,
                   user_id,
                   brand,
                   system_id,
                   energy_supplier
        FROM `nox-energy-431214.asset_telemetry_raw.rbc_decisions`
        WHERE partition_date >= DATE('{partition_date_start}')
            AND partition_date <= DATE('{partition_date_end}')
            AND energy_supplier != '{EXCLUDED_ENERGY_SUPPLIER}'
        ORDER BY brand, device_id
    """
    query_job = get_big_query_client().query(query)
    response = query_job.result()
    data = []
    for row in response:
        data.append(dict(row))
    return data

In [50]:
devices = get_unique_steered_devices_for_period(start_period, end_period)
devices = [SteeredDevice(**d) for d in devices]
for d in devices:
    d.dates = []

In [51]:
for i in range(7):
    _date = start_period.shift(days=i)
    day_devices = get_unique_steered_devices_for_day(_date)
    for d in devices:
        if d.device_id in list(day_devices.keys()):
            d.dates.append(_date)


In [54]:
for d in devices:
    print(d.device_id, len(d.dates))

3a9c13bc-fbdd-5823-92dd-1f6b6d64ac9c 7
3a9c13bc-fbdd-5823-92dd-1f6b6d64ac9c 7
58d1acbe-c64b-5ac1-910d-9cd64cb41699 7
58d1acbe-c64b-5ac1-910d-9cd64cb41699 7
238d79dd-740c-497e-8ec2-60929f06ff68 7
3bbe62f5-9cd6-41d2-b18b-b17a62df407b 7
53812aa8-8b0b-48f9-a451-12007551e29e 7
6bb416ea-f469-46c5-8b8e-5df6b3a3fed3 7
73eae5a0-34a2-42ad-9d3c-eb00489d5011 7
74d40dc8-f53c-4c66-9245-20ee8a6947f1 7
df9a5c3c-b733-4b30-a6de-f614cb662a08 7
e8e2a998-8847-47f4-b7e5-1b729e0f315a 7
130895748 7
137079825 7
1536949 7
4879600 7
4879869 7
1a6b47d4-a957-51ad-bb83-138e8e9f0ed8 7
374ffb72-af5b-518a-91ea-e0ef1028d96b 7
3c77a309-92da-5597-8a9c-e6668b3b51a9 1
612e7767-0696-56e8-92b5-f44fca42a4b2 7
6a852169-c293-526e-991b-c93cf85f1987 7
70fa60c9-42ac-574f-9479-5e0d44a74ec3 7
80e24c49-5726-5860-9c43-e0dfdfa126d1 7
a79c7133-1bf2-53e1-8cfa-7f1c4ee5d558 6
aa3a21f9-1dcf-53a8-8754-3085f99990a8 7
b36e0feb-ca3d-537e-a6d6-469d15f9f28e 7
7571381726530104 7
7637415031272222 7
7637415043115241 1
7637415045896193 7
773617204625

In [55]:
len(devices)

34

In [58]:
len(list(filter(lambda d: len(d.dates) == 7, devices)))

31

In [ ]:
print(devices)